In [47]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from rasterio import features, open
from rasterio.transform import from_origin, rowcol
from rasterio.plot import show
import rasterstats as rs
import rasterio
from shapely.geometry import box


In [48]:
def read_ref_raster(ref_raster_name):
    with rasterio.open(ref_raster_name) as ref:
        ref_transform = ref.transform
        ref_crs = ref.crs
        ref_width = ref.width
        ref_height = ref.height
    
    return (ref_transform, ref_crs, ref_width, ref_height)

def write_raster(output_name, raster, ref_height, ref_width, ref_crs, ref_transform):
    with rasterio.open(
        output_name,
        "w",
        driver="GTiff",
        height=ref_height,
        width=ref_width,
        count=1,
        dtype=raster.dtype,
        crs=ref_crs,
        transform=ref_transform,
        nodata=0
    ) as dst:
        dst.write(raster, 1)

In [49]:
def read_shapefile(shapefile_name, ref_crs):
    shapefile =  gpd.read_file(shapefile_name)
    return shapefile.to_crs(ref_crs)

def rasterize_binary(ref_raster_name, shapefile, output_name):        
    (ref_transform, ref_crs, ref_width, ref_height) = read_ref_raster(ref_raster_name)
    
    shapes = ((geom, 1) for geom in shapefile.geometry)
    raster = features.rasterize(
        shapes,
        out_shape=(ref_height, ref_width),
        transform=ref_transform,
        fill=0,
        dtype=np.uint8
    )
    
    write_raster(output_name, raster, ref_height, ref_width, ref_crs, ref_transform)

In [56]:
def pixel_bbox(row, col, ref_transform):
    px = ref_transform.a   
    py = ref_transform.e   

    x_left = ref_transform.c + col * px
    y_top = ref_transform.f + row * py

    x_right = x_left + px
    y_bottom = y_top + py

    return box(
        min(x_left, x_right),
        min(y_bottom, y_top),
        max(x_left, x_right),
        max(y_bottom, y_top),
    )

def rasterize_interpolated(ref_transform, ref_crs, ref_width, ref_height, gdf, attribute_name, output):
    raster = np.zeros((ref_height, ref_width), dtype=np.float32)

    print("iterating over ", gdf.length, " rows")
    for _, feat in gdf.iterrows():
        geom = feat.geometry
        value = feat[attribute_name]
        
        if(np.isnan(value) or geom.area == 0): 
            continue

        minx, miny, maxx, maxy = geom.bounds

        row_min, col_min = rowcol(ref_transform, minx, maxy)
        row_max, col_max = rowcol(ref_transform, maxx, miny)

        row_min = max(0, row_min)
        col_min = max(0, col_min)
        row_max = min(ref_height - 1, row_max)
        col_max = min(ref_width - 1, col_max)

        for row in range(row_min, row_max + 1):
            for col in range(col_min, col_max + 1):
                pixel = pixel_bbox(row, col, ref_transform)
                inter_area = geom.intersection(pixel).area

                if inter_area > 0:
                    raster[row, col] += value * (inter_area / geom.area)
                    print(row, col, raster[row, col], value, inter_area, geom.area)

    print("writing out raster")
    write_raster(output, raster, ref_height, ref_width, ref_crs, ref_transform)

In [46]:
ref_transform, ref_crs, ref_width, ref_height = read_ref_raster("./hague/hague_light.tif")
shapefile = read_shapefile("./hague/building_volumes/hague_building_heights.shp", ref_crs)

In [54]:
feat = shapefile.iloc[1]
print(feat.fid)
geom = feat.geometry
minx, miny, maxx, maxy = geom.bounds

row_min, col_min = rowcol(ref_transform, minx, maxy)
row_max, col_max = rowcol(ref_transform, maxx, miny)

raster = np.zeros((ref_height, ref_width), dtype=np.float32)


print(row_min, col_min, row_max, col_max)

row_min = max(0, row_min)
col_min = max(0, col_min)
row_max = min(ref_height - 1, row_max)
col_max = min(ref_width - 1, col_max)

value = feat['rf_volume_']

for row in range(row_min, row_max + 1):
    for col in range(col_min, col_max + 1):
        pixel = pixel_bbox(row, col, ref_transform)
        inter_area = geom.intersection(pixel).area
        print(inter_area, col, row)
        
        if inter_area > 0:
            raster[row, col] += value * (inter_area / feat.geometry.area)
            print(raster[row, col])

write_raster('test.tif', raster, ref_height, ref_width, ref_crs, ref_transform)

13212482.0
55 59 55 59
5.998648999936753 59 55
15.270629


CPLE_AppDefinedError: Deleting test.tif failed: Permission denied

In [55]:
rasterize_interpolated(ref_transform, ref_crs, ref_height, ref_width, shapefile, "rf_volume_", "hague_building_volumes.tif")

iterating over  0         13.972241
1          9.998987
2         34.799918
3         15.936352
4         10.260249
            ...    
712892    33.163097
712893    12.609992
712894    29.561026
712895    33.318785
712896    12.596071
Length: 712897, dtype: float64  rows
54 59 29.207502 29.207502365112305 11.953696999943496 11.953696999943494
55 59 15.270629 15.270628929138184 5.998648999936753 5.998648999936752
55 59 715.99725 700.7266235351562 69.25515400014747 69.25515400014747
55 59 1416.7239 700.7266235351562 7.595372000099759 7.59537200009976
55 59 1433.1337 16.409832000732422 6.347278000057186 6.347278000057186
54 59 46.416298 17.20879364013672 6.273547500031404 6.2735475000314045
55 59 1886.1089 452.97515869140625 55.61019900024908 55.61019900024908
55 60 469.72354 469.7235412597656 57.223922500036664 57.22392250003666
55 68 446.63687 446.6368713378906 54.518706000235014 54.51870600023501
55 68 473.75143 27.114574432373047 10.298791500003716 10.298791500003716
55 68 499.3459 2

KeyboardInterrupt: 